In [ ]:
import pandas as pd

from kafka import KafkaConsumer, KafkaProducer
import json
import uuid
import pandas as pd
import csv
import time
from datetime import datetime

In [ ]:
server1 = 'broker1:9093'
server2 = 'broker2:9095'
server3 = 'broker3:9097'

topic_perf = "performance"

In [ ]:
consumer_perf = KafkaConsumer(topic_perf,
                         auto_offset_reset='earliest',
                         bootstrap_servers=[server2],
                         api_version=(0, 10),
                         value_deserializer = json.loads,
                         #consumer_timeout_ms=50,
                         client_id="performance")

def consume_perf(consumer, topic_name):
    # get data from Kafka
    messages = []
    for msg in consumer:
        messages.append(msg.value)

        print(f'{datetime.now()}: {len(msg.value)} messages received on topic: {topic_name}', flush=True)

    # save received data to csv
    if len(messages) != 0:
        # print  latest received data point
        print(f'latest data: {messages[-1]["data"]}', flush=True)
        print(flush=True)
    else:
        print('no new messages - waiting for producer', flush=True)


In [ ]:
consume_perf(consumer_perf, topic_perf)